In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import train,prepare,evaluate,tune
from models.Interactors import FIM_Interactor, KNRM_Interactor
from models.SFI import SFI_gating, SFI_gating_MultiView
from configs.ManualConfig import hparams

In [6]:
name='sfi'
hparams['k'] = 30
hparams['select'] = 'gating'
hparams['onehot'] = True
# hparams['threshold'] = 0.5

In [7]:
# hparams['validate'] = True
vocab, loaders = prepare(hparams, pin_memory=False)

[2021-04-18 10:34:39,731] INFO (root) Hyper Parameters are
{'scale': 'demo', 'mode': 'train', 'batch_size': 10, 'title_size': 20, 'abs_size': 40, 'his_size': 50, 'vert_num': 18, 'subvert_num': 293, 'npratio': 4, 'dropout_p': 0.2, 'query_dim': 200, 'embedding_dim': 300, 'filter_num': 150, 'value_dim': 16, 'head_num': 16, 'epochs': 8, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'device': 'cpu', 'attrs': ['title'], 'k': 30, 'select': 'gating', 'save_step': [0], 'news_id': False, 'validate': False, 'interval': 10, 'spadam': True, 'onehot': True}
[2021-04-18 10:34:39,733] INFO (root) preparing dataset...
[2021-04-18 10:34:43,734] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [8]:
record = next(iter(loaders[0]))

In [11]:
from models.Encoders.FIM import FIM_Encoder

encoder = FIM_Encoder(hparams, vocab)
interactor = FIM_Interactor()
# interactor = KNRM_Interactor()

hparams['name'] = '-'.join([name,encoder.name,hparams['select']])

# sfi = SFI_gating_MultiView(hparams, encoder, interactor).to(hparams['device'])
sfi = SFI_gating(hparams, encoder, interactor).to(hparams['device'])

# sfi.load_state_dict(torch.load('data/model_params/sfi-fim-fim-gating/large_epoch2_step16916_[hs=50,topk=50,contra=0,attrs=title].model', map_location=hparams['device'])['model'])

In [12]:
sfi(record)

tensor([[-1.4493, -1.6813, -1.6532, -1.6305, -1.6509],
        [-1.8871, -1.4121, -1.8748, -1.6347, -1.3608],
        [-2.0034, -1.2797, -1.8911, -1.6376, -1.4203],
        [-1.6491, -1.5123, -2.1530, -1.0962, -1.9870],
        [-1.4529, -1.6627, -1.6071, -1.5913, -1.7582],
        [-1.7003, -1.9281, -1.7101, -1.3364, -1.4770],
        [-0.6197, -2.0016, -1.9635, -2.9857, -1.9961],
        [-2.0064, -1.6496, -1.4424, -1.6374, -1.4165],
        [-1.2671, -1.8530, -1.8601, -1.4277, -1.7953],
        [-2.7425, -1.4901, -2.1843, -0.9616, -1.5352]],
       grad_fn=<LogSoftmaxBackward>)

In [ ]:
sfi = tune(sfi, hparams, loaders)

In [ ]:
evaluate(sfi, hparams, loaders[1], loading=True)